In [101]:
import pandas as pd
import gradio as gr
from sklearn.linear_model import LinearRegression

## Load dataset

In [ ]:
# 1. Setup for Linear Model

# Data Source: Global AI Workforce & Automation 2015-2025 https://www.oxfordinsights.com/ai-readiness-index-2025

features = ['AI_Investment_BillionUSD', 'Reskilling_Investment_MillionUSD', 
            'AI_Policy_Index', 'Automation_Rate_Percent', 'Productivity_Index']
target = 'AI_Readiness_Score'

# Loads dataset
# Note: Ensure 'global_ai_workforce_automation_2015_2025.csv' is in the folder
try:
    df = pd.read_csv('global_ai_workforce_automation_2015_2025.csv') #Dataset path
    X = df[features]
    y = df[target]
    model = LinearRegression() 
    model.fit(X, y)
except FileNotFoundError:
    print("Dataset not found.")

## Logic & Process

In [103]:

# 2. Prediction Logic with "Artificial Country" Tier
def predict_readiness(investment, reskilling, policy, automation, productivity):
    # Predicts AI readiness. Modified to allow 'Super-Scaling' for values > 90.

    # input dataframe
    input_data = pd.DataFrame([[investment, reskilling, policy, automation, productivity]], 
                              columns=features)
    
    # Calculate score (Allowing it to hit 100)
    raw_prediction = model.predict(input_data)[0]
    final_score = min(max(raw_prediction, 0), 100.0)
    
    # Benchmarking 2025 Strategic Tiers ]
    # Informations are based by Oxford Insights AI Readiness Index 2025
    if final_score >= 90:
        # NOTE: scores above 90 are classified as "Artificial Country" below <89 are already the 2025 maximum charts. This a unrecorded fictional tier.
        status = "ARTIFICIAL COUNTRY (Beyond USA/Singapore)"
    elif final_score >= 80:
        # TIER 1: Current Global Leaders (ex: USA, Signapore)
        status = "GLOBAL FRONTIER (World Leading!)"
    elif final_score >= 60:
        # TIER 2: High Potential based on select 1st world countries (ex: UAE, Germany, Japan)
        status = "1st World Ready"
    elif final_score >= 40:
        # TIER 3: Developing Economies (ex: India, Brazil)
        status = "Developing & Global Average"
    else:
        # TIER 4: Early Stage commonly around the world
        status = "Early Stages"
        
    return f"{final_score:.2f} / 100", status

## Gradio Visualization

In [104]:
# 3. Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# National AI Strategic Simulator 2025 or beyond reality")
    gr.Markdown("Adjust the sliders values below. \n\n Based on [Oxford Insights AI Readiness Index 2025](https://www.oxfordinsights.com/ai-readiness-index-2025)")
    
    with gr.Row():
        with gr.Column():
            # AI Investment Slider
            inv = gr.Slider(0, 600, value=50, label="AI Investment (Billion USD)", 
                           info="National investment to AI technologies")
            
            # National Reskilling Slider    
            resk = gr.Slider(0, 20000, value=500, label="Reskilling (Million USD)", 
                            info="National Workforce Overhaul. **10,000+ for maximum population impact or double it.**")
            
            # National AI POLICY INDEX
            pol = gr.Slider(0, 1.5, value=0.5, step=0.01, label="AI Policy Index", 
                           info="Regulation Value. **Set to 1.0 for max national regulation or 1.1+ for global mandatation.**")
            
            # Standard Metrics
            # Automation Rate Slider
            auto = gr.Slider(0, 100, value=25, label="Automation Rate (%)", 
                            info="AI integration and replacement in workspaces percenatages.")
            # Productivity Index Slider
            prod = gr.Slider(0, 100, value=50, label="Productivity Index", 
                            info="GDP per capita efficiency rate.")
            
            btn = gr.Button("Run Simulation", variant="primary")
         
         # Output   
        with gr.Column():
            score_out = gr.Label(label="Predicted Readiness Score")
            status_out = gr.Textbox(label="Geopolitical Classification")

    btn.click(fn=predict_readiness, inputs=[inv, resk, pol, auto, prod], outputs=[score_out, status_out])

demo.launch()

* Running on local URL:  http://127.0.0.1:7885
* To create a public link, set `share=True` in `launch()`.
